<a href="https://colab.research.google.com/github/anitaokoh/translating_transcribing_youtube_video/blob/main/creating_and__inputing_a_subtitle_for_youtube_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating and Inputing a subtitle for Youtube video



### Install the libraries, dependencies needed and import the libriaries

In [1]:
!pip install moviepy
!pip install pytube
!apt install imagemagick
!cat /etc/ImageMagick-6/policy.xml | sed 's/none/read,write/g'> /etc/ImageMagick-6/policy.xml
!pip install srt
!pip install git+https://github.com/openai/whisper.git
!pip install -U kora

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
imagemagick is already the newest version (8:6.9.11.60+dfsg-1.3ubuntu0.22.04.3).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-63oad66o
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-63oad66o
  Resolved https://github.com/openai/whisper.git to commit 8bc8860694949db53c42ba47ddc23786c2e02a8b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
from pytube import YouTube
# from moviepy.editor import *
# from moviepy.editor import VideoFileClip
import whisper
import srt
import datetime
from moviepy.editor import VideoFileClip, CompositeVideoClip,  TextClip
from moviepy.video.tools.subtitles import SubtitlesClip
from kora.drive import upload_public

###  Creating a Subtitled Video

In [3]:
#Step 1: Download the YouTube Video of choice
url = 'https://www.youtube.com/watch?v=lyYBfMqECIY&t=2s'
yt = YouTube(url)
video = yt.streams.filter(progressive=True, file_extension='mp4').first()
video.download(filename='youtube_video.mp4')

'/content/youtube_video.mp4'

In [4]:


# Step 2: Extract Audio from Video
video = VideoFileClip("/content/youtube_video.mp4")
audio = video.audio
audio.write_audiofile("extracted_audio.wav")




#Step 3: Transcribe and Translate Audio
model = whisper.load_model("base")
result = model.transcribe('extracted_audio.wav', task="translate", language="de")

#Step4: Create SRT Subtitles
subtitles = []
start_time = 0
for segment in result["segments"]:
    start = datetime.timedelta(seconds=segment["start"])
    end = datetime.timedelta(seconds=start_time + segment["end"])
    subtitles.append(srt.Subtitle(index=len(subtitles) + 1, start=start, end=end, content=segment["text"]))
srt_content = srt.compose(subtitles)
with open("subtitles.srt", "w") as f:
    f.write(srt_content)
print("Subtitles created successfully.")


MoviePy - Writing audio in extracted_audio.wav


MoviePy - Done.


  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



Subtitles created successfully.


In [5]:
#Step 5: Add the subtitle back into the video
def create_subtitle_clip(srt_file):
    def generator(txt):
        return TextClip(txt, font='Arial', fontsize=15, color='white')

    return SubtitlesClip(srt_file, generator)

video = VideoFileClip("/content/youtube_video.mp4")
subtitles = create_subtitle_clip("subtitles.srt")

# Overlay subtitles on the original video
video_with_subs = CompositeVideoClip([video, subtitles.set_position(('center', 'bottom'))])

# # Write the result to a file
video_with_subs.write_videofile("video_with_subtitles.mp4",codec="libx264", audio_codec='aac')


Moviepy - Building video video_with_subtitles.mp4.
MoviePy - Writing audio in video_with_subtitlesTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video video_with_subtitles.mp4



t: 100%|█████████▉| 20027/20036 [04:52<00:00, 98.67it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/youtube_video.mp4, 691200 bytes wanted but 0 bytes read,at frame 20034/20036, at time 801.36/801.41 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready video_with_subtitles.mp4


In [6]:
# Optional Step 6: View the video subtitled

url = upload_public('/content/video_with_subtitles.mp4')
# then display it
from IPython.display import HTML
HTML(f"""<video src={url} width=500 controls/>""")